In [17]:
library(vegan)
library(ggplot2)
library(tidyverse)
library(MicrobeR)

ERROR: Error in library(MicrobeR): there is no package called ‘MicrobeR’


In [2]:
bc=read.csv('braycurtis-distance-matrix.tsv',sep='\t',row.names=1,check.names=FALSE)
jaccard=read.csv('beta-diversity-metrics/jaccard-distance-matrix.tsv',sep='\t',row.names=1,check.names=FALSE)
unweightUF=read.csv('beta-diversity-metrics/unweighted_unifrac-distance-matrix.tsv',sep='\t',row.names=1,check.names=FALSE)
weightUF=read.csv('beta-diversity-metrics/weighted_unifrac-distance-matrix.tsv',sep='\t',row.names=1,check.names=FALSE)

In [3]:
dim(bc)

[1] 1059 1059

In [4]:
# head(bc,2)

,SRR21105746,SRR6110936,SRR5559431,SRR5559472,SRR15720152,SRR5559561,SRR5559709,ERR4586659,SRR18270619,SRR15560942,⋯,SRR21105889,SRR15720113,SRR5559704,SRR21105663,SRR21105661,SRR5559396,ERR4296678,SRR21105769,SRR15719945,SRR5559429
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SRR21105746,0.0000000,0.3847364,0.4850511,0.2866135,0.4816230,0.2956615,0.3264584,0.7707654,0.3575362,0.4529617,⋯,0.5481061,0.4568394,0.4972463,0.518096,0.8794537,0.2874002,0.5728897,0.5814881,0.3849612,0.5261886
SRR6110936,0.3847364,0.0000000,0.3876588,0.2591885,0.4958975,0.1956839,0.2188940,0.7091716,0.2354164,0.5600202,⋯,0.4244689,0.5338316,0.3940654,0.527144,0.8819827,0.2627852,0.5002810,0.5552433,0.4466674,0.4294706


In [5]:
md <- read.csv('metadata.txt',
               row.names=1, sep='\t', check.names=FALSE) 
# head(md,2)

,sample_type,project_ID,sequencing_region,diagnose,age,gender,BMI,tumor_site,T_stage,N_stage,⋯,sequencing_region_merge,host_replicates,treatment_yn,replicate_2,HNSCC,_sample_id,_calour_original_abundance,observed_features,shannon_entropy,faith_pd
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>
SRR21105746,Oral_rinse,PRJNA870048,V3-V4,OSCC,63,M,NA,tongue,,,⋯,V3-V4,,,,,SRR21105746,48812,184,3.804584,16.04912
SRR6110936,Saliva,PRJNA412445,V4-V5,OSCC,63,F,NA,,II,0,⋯,V4-V5,,,,,SRR6110936,113189,177,4.731407,18.01963


In [6]:
table(md$project_ID)


  OEP000837  PRJEB37501  PRJEB39064 PRJNA386665 PRJNA412445 PRJNA421234 
         44          73          73         374          17          21 
PRJNA666891 PRJNA700849 PRJNA744870 PRJNA751046 PRJNA756784 PRJNA813634 
         12          10         152          18          59          25 
PRJNA870048 
        181 

In [7]:
beta_stat <- function(p,dis,c1,c2){
    As <- list()
    s_md <- subset(md,project_ID==p) %>% subset(diagnose==c1 | diagnose==c2)
    sample_ids=intersect(row.names(s_md),row.names(dis))
    s_md <- s_md[sample_ids,]
    distance <- dis[sample_ids,sample_ids]
    
    adnonis_res <- adonis(distance ~ diagnose, data = s_md, permutations = 9999)$aov.tab
    return(adnonis_res)
}

# healthy_control vs OSCC

In [8]:
projects=c('OEP000837','PRJEB39064','PRJNA386665','PRJNA421234','PRJNA700849','PRJNA744870',
          'PRJNA751046','PRJNA756784','PRJNA870048')

## calculation within each dataset

In [9]:
distance_metric <- list(bc,jaccard,weightUF,unweightUF)
distance_str <- c('Bray Curtis','Jaccard','Weighted UniFrac','Unweighted UniFrac')

In [10]:
set.seed(1)
PerStudy_beta<-list()
for (i in 1:length(distance_metric)){
    dis <- distance_metric[[i]]
    dis_str <- distance_str[i]

    for(p in projects){        
        PerStudy_beta[[dis_str]][[p]]<-beta_stat(p,dis,'Healthy_control','OSCC')
   }
}

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' wil

In [14]:
# PerStudy_beta$`Bray Curtis`$OEP000837

In [11]:
data <- data.frame()

for (dis_str in distance_str){
   diag_res <- data.frame(row.names=projects) 
   for (p in projects){       
       diag_res[p,1] <- PerStudy_beta[[dis_str]][[p]]['diagnose','R2']
       diag_res[p,2] <- PerStudy_beta[[dis_str]][[p]]['diagnose','Pr(>F)'] 
   }   
   names(diag_res) <- c('R2','pvalue')
   diag_res['study']<-row.names(diag_res)
   diag_res['metric'] <- dis_str
   data <- rbind(data, diag_res)
}

In [16]:
# data

## combined results

In [12]:
# combined 
mtd <- subset(md,project_ID %in% projects) %>% 
subset(diagnose == 'Healthy_control'| diagnose == 'OSCC')

df <- data.frame()
for (i in 1:length(distance_metric)){
    dis <- distance_metric[[i]]
    dis_str <- distance_str[i]
    
    sample_ids = intersect(row.names(mtd),row.names(dis))
    s_md <- mtd[sample_ids,]
    distance <- dis[sample_ids,sample_ids]
    adnonis_res <- adonis(distance ~ diagnose, permutations = 9999,
                          strata=s_md$project_ID,data = s_md)$aov.tab
    R2 <- adnonis_res['diagnose','R2']
    pvalue <- adnonis_res['diagnose','Pr(>F)']
    l <- data.frame('R2'=R2,'pvalue'=pvalue,'study'='Combined','metric'=dis_str)
    df <- rbind(df, l)
    
}

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead

'adonis' will be deprecated: use 'adonis2' instead



In [13]:
hg <- rbind(data,df) %>% mutate(Compare='Healthy control versus OSCC') %>% 
mutate(Pvalue=case_when(
    pvalue < 0.05 ~ 'P < 0.05',  
    TRUE~"ns"))

## visualization

In [14]:
cols <- c("#E69F00",
         "#0072B2",
         "#D6604D",
         "#7570B3")

In [15]:
ggplot(data = hg, aes(x = R2, y =study, color=metric)) + 
geom_point(aes(shape =Pvalue),size=2) +
theme_MicrobeR() +
theme(panel.border = element_blank(), axis.line = element_line())+ 
theme(axis.text.x = element_text(angle = 30, hjust = 1), 
        panel.grid = element_blank(), 
        axis.text = element_text(size = 10,colour = 'black'), 
        axis.title = element_text(size = 10),
        plot.title = element_text(size = 10,hjust = 0.5),
      legend.title = element_text(size = 10),
      legend.text = element_text(size = 8),
     plot.margin=unit(rep(2,4),'lines'))+scale_color_manual(values=cols)+
labs(title = 'Healthy control vs OSCC',y='') #
# ggsave("figures-and-tables/adonis-healthy-control-vs-OSCC.pdf", 
#        device="pdf", height=5, width=5.2, useDingbats=F)

ERROR: Error in theme_MicrobeR(): could not find function "theme_MicrobeR"
